In [35]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "I am a student majoring in computer science"
encoded_input = tokenizer(text, return_tensors='pt',)
output = model(**encoded_input,output_attentions=True)
output.keys()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


odict_keys(['last_hidden_state', 'pooler_output', 'attentions'])

In [36]:
output.attentions[-1][0].shape

torch.Size([12, 11, 11])

In [37]:
output.attentions[-1][0][0]

tensor([[0.1634, 0.0378, 0.1396, 0.0279, 0.0160, 0.0101, 0.0033, 0.0046, 0.0285,
         0.0270, 0.5416],
        [0.0253, 0.0120, 0.0235, 0.0158, 0.0201, 0.0116, 0.0069, 0.0042, 0.0116,
         0.0155, 0.8535],
        [0.0145, 0.0050, 0.0116, 0.0074, 0.0094, 0.0050, 0.0028, 0.0033, 0.0064,
         0.0107, 0.9240],
        [0.0073, 0.0056, 0.0089, 0.0098, 0.0173, 0.0127, 0.0058, 0.0087, 0.0135,
         0.0169, 0.8934],
        [0.0239, 0.0039, 0.0085, 0.0147, 0.0168, 0.0187, 0.0108, 0.0121, 0.0219,
         0.0397, 0.8290],
        [0.0251, 0.0048, 0.0116, 0.0130, 0.0226, 0.0203, 0.0090, 0.0093, 0.0198,
         0.0589, 0.8057],
        [0.0127, 0.0024, 0.0060, 0.0084, 0.0236, 0.0194, 0.0049, 0.0082, 0.0089,
         0.0219, 0.8836],
        [0.0185, 0.0052, 0.0068, 0.0135, 0.0253, 0.0339, 0.0105, 0.0206, 0.0359,
         0.0674, 0.7625],
        [0.0104, 0.0025, 0.0039, 0.0048, 0.0046, 0.0046, 0.0021, 0.0081, 0.0114,
         0.0152, 0.9324],
        [0.0302, 0.0097, 0.0226, 0.01

In [38]:
text

'I am a student majoring in computer science'

In [39]:
encoded_input

{'input_ids': tensor([[ 101, 1045, 2572, 1037, 3076, 2350, 2075, 1999, 3274, 2671,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [40]:
tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])

['[CLS]',
 'i',
 'am',
 'a',
 'student',
 'major',
 '##ing',
 'in',
 'computer',
 'science',
 '[SEP]']

In [3]:
def merge_sort(ls):
    print(ls)
    if len(ls)<=1:
        return ls
    mid = len(ls)//2
    ls1 = merge_sort(ls[0:mid])
    ls2 = merge_sort(ls[mid:])
    print(ls,ls1,ls2)
    ls3 = []
    i = 0
    j = 0
    while i<=len(ls1)-1 and j<=len(ls2)-1:
        if ls1[i]>ls2[j]:
            ls3.append(ls2[j])
            j += 1
        else:
           ls3.append(ls1[i])
           i += 1
    return ls3

ls4 = merge_sort([6542, 6570, 5709, 3794, 5128, 6093, 2349, 8164, 8144, 139])
print(ls4)

[6542, 6570, 5709, 3794, 5128, 6093, 2349, 8164, 8144, 139]
[6542, 6570, 5709, 3794, 5128]
[6542, 6570]
[6542]
[6570]
[5709, 3794, 5128]
[5709]
[3794, 5128]
[3794]
[5128]
[6093, 2349, 8164, 8144, 139]
[6093, 2349]
[6093]
[2349]
[8164, 8144, 139]
[8164]
[8144, 139]
[8144]
[139]
[139]
